# **Create or Update Seafloor Site Metadata File**

This notebook goes through the steps necessary to update or create a new site metadata file.

***
## **1. Import packages**

In [ ]:
from datetime import datetime
import json
import ipywidgets as widgets
from IPython.display import display
import functools
import os

from es_sfgtools.utils.metadata.site import Site, import_site, ArrayCenter, TopLevelSiteGroups, SubLevelSiteGroups
from es_sfgtools.utils.metadata.utils import convert_custom_objects_to_dict
from es_sfgtools.utils.metadata.benchmark import Location
from es_sfgtools.utils.archive_pull import download_file_from_archive
from es_sfgtools.utils.metadata.campaign import campaign_checks

***
## **2. Choosing the appropriate site metadata file**

Using the options below, you may either `create` or `update` a site metadata file. If the site already exists in the earthscope archive, the metadata file should be accessible on https://gage-data.earthscope.org/archive/seafloor. 

Only choose 1 of the 3 options.

### **Option 1:** Create a new site file 

In [ ]:
# ----------------------- Update these values ----------------------- 
# Add site names and networks
names = ['']                        # list of names used for this site, including the 4 char ID, eg. ['AAAA']
networks = ['']                     # list of networks this site is a part of, eg. ['aleutian']          

# Add time of origin (Benchmarks drop date)
time_of_origin = ''                   # format '1900-01-01T00:00:00'

local_geoid_height = 0

# Add array center coordinates
x = 0
y = 0
z = 0

# ---------------------- Do not update code below ---------------------
site_class = Site(names=names, networks=networks, timeOrigin=time_of_origin, localGeoidHeight=local_geoid_height, arrayCenter=ArrayCenter(x=x, y=y, z=z))
print("Created site object\n")
site_class.print_json()

### **Option 2:** Load existing site metadata from local file path
Use the cell below to load an existing site metadata file to update

In [ ]:
# Pull in site metadata from json file
# ----------------------- Update these values -----------------------
json_file_path = 'SITE.json'        # JSON file path

# ---------------------- Do not update code below ---------------------
site_class = import_site(json_file_path)
site_class.print_json()

### **Option 3:** Pull existing site metadata directly from GAGE data
Use the cell below to load an existing site metadata file from the GAGE data archive

In [ ]:
# ----------------------- Update these values -----------------------
gage_http_file_link = ''
destination_directory = './gage_data'   # directory where the files will be downloaded to (default is the current directory/gage_data)
token_path = '.'                        # directory where the token will be stored (default is the current directory)


# ---------------------- Do not update code below ---------------------
download_file_from_archive(url=gage_http_file_link, 
                           dest_dir=destination_directory)

file_name = os.path.basename(gage_http_file_link)
site_class = import_site(os.path.join(destination_directory, file_name))
site_class.print_json()

## **3. Site additions and updates**
***
Below are optional additions and updates you can make to the site metadata file. You do not need to go in order to update the file. Options include..
* Required site specific metadata
   * Benchmarks 
   * Transponders
* Campaigns
   * Basic campaign information
   * Surveys within each campaign
* Other site data
   * Reference Frames
   * Additional Transponder Information

#### **Instructions**
1. Input values in __*Update these Values*__ section
   * For a new section, enter all known values, including a start and end date. If a start and end date is not changed from "1900", then it will not be added into the json
   * For updating an existing section, update only the values you wish to change. Leave all other fields empty or as they are.
2. Run the cell and check the printed JSON input is correct.
3. Use the cell below to add, update or delete.

If you need to see the site class object at any time in the process, scroll to the **Write out JSON file** section at the bottom and run the cell that checks the output. 


### **Come back and view the full site output at any time using the cell below**

In [ ]:
site_class.print_json()


## **Required site specific metadata**
***


## Benchmarks

If you are updating a site file that already exists, you will only need to update or add a new *Benchmark* as needed.

* **New Benchmark:** please enter the required information below and then run the cell, check your output and set `ADD_NEW_BENCHMARK` to `True` in the next cell prior to running. 

* **Updating a Benchmark:**, please only enter the values you wish to update including the `benchmark_name` you wish to update. All previous values in the site file will be kept if not updated. One entered, run the cell, check your output and set `UPDATE_EXISTING_BENCHMARK` to `True` in the next cell prior to running. 

* **Deleting a Benchmark:** When deleting, only the `benchmark_name` is needed. Once entered, run the cell, and set `DELETE_BENCHMARK` to `True` in the next cell prior to running. 

In [ ]:
benchmark={} 
# ----------------------- Update these values ----------------------- 

# -- Required information for new benchmark -- 
benchmark['name'] = ""                      # "IVB1-1" - REQUIRED
benchmark['benchmarkID'] = ""               # stamped on name

aPrioriLocation = Location(
    latitude = 0,                           # latitude of benchmark drop point
    longitude = 0,                          # longitude of benchmark drop point
    elevation = 0                           # elevation of benchmark drop point
)

benchmark['start'] = ''                     # Start date, Format: 1900-01-01T00:00:00

# -- Optional Benchmark information --
benchmark['end'] = ''                       # End date, Format 1900-01-01T00:00:00'

# Drop points
dropPointLocation = Location(
    latitude = 0,                               # latitude of benchmark drop point
    longitude = 0,                              # longitude of benchmark drop point
    elevation = 0                               # elevation of benchmark drop point
)

# ----------------------- Do not update code below ----------------------- 
benchmark['aPrioriLocation'] = aPrioriLocation
benchmark['dropPointLocation'] = dropPointLocation

print(json.dumps(convert_custom_objects_to_dict(benchmark),indent=2))

#### Choose whether to add new, update, or delete the benchmark you defined above

In [ ]:
# Set only one of the following to True
ADD_NEW_BENCHMARK = True                
UPDATE_EXISTING_BENCHMARK = False
DELETE_BENCHMARK = False

# ----------------------- Do not update code below ----------------------- 
site_class.run_component(component_type=TopLevelSiteGroups.BENCHMARKS, component_metadata=benchmark, 
                        add_new=ADD_NEW_BENCHMARK, update=UPDATE_EXISTING_BENCHMARK, delete=DELETE_BENCHMARK)

## Benchmark Transponders

If you are updating a site file that already exists, you will only need to update or add a new *Transponder* as needed.

* **New Transponder:** please enter the required information below and then run the cell, check your output and set `ADD_NEW_TRANSPONDER` to `True` in the next cell prior to running. 

* **Updating a Transponder:**, please only enter the values you wish to update including the `transponder_benchmark_name` and `transponder_address` you wish to update. All previous values in the site file will be kept if not updated. One entered, run the cell, check your output and set `UPDATE_EXISTING_TRANSPONDER` to `True` in the next cell prior to running. 

* **Deleting a Transponder:** When deleting, only the `transponder_benchmark_name` and `transponder_address` is needed. Once entered, run the cell, and set `DELETE_TRANSPONDER` to `True` in the next cell prior to running. 

In [ ]:
transponder = {}
# ----------------------- Update these values ----------------------- 

# -- Required information for new transponder --
transponder_benchmark_name = benchmark['name']       # e.g "IVB1-1" -  Change this if you didn't previously add a benchmark or wish to add to a different benchmark
transponder['address'] = ""                         # e.g 5209
transponder['start'] = ""                           # Start date of transponder, format '1900-01-01T00:00:00'
transponder['tat'] = ""                             # TAT in ms


# -- Optional transponder information --
transponder['uid'] = ""                             # UID of transponder
transponder['model'] = ""                           # transponder model number
transponder['serialNumber'] = ""                    # transponder serial number
transponder['batteryCapacity'] = ""                 # e.g 4 Ah

transponder['end'] = ""                            # End date of transponder, format '1900-01-01T00:00:00'

transponder['notes'] = ""                           # Any notable events

# ----------------------- Do not update code below ----------------------- 
print(json.dumps(transponder, indent=2))

#### Choose whether to add new, update, or delete the transponder you defined above

In [ ]:
ADD_NEW_TRANSPONDER = True
UPDATE_EXISTING_TRANSPONDER = False
DELETE_TRANSPONDER = False

# ----------------------- Do not update code below ----------------------- 
site_class.run_sub_component(component_type=TopLevelSiteGroups.BENCHMARKS, component_name=transponder_benchmark_name,
                            sub_component_type=SubLevelSiteGroups.TRANSPONDERS, sub_component_metadata=transponder,
                            add_new=ADD_NEW_TRANSPONDER, update=UPDATE_EXISTING_TRANSPONDER, delete=DELETE_TRANSPONDER)

***
## Campaign Data
***
### Campaign information

* **New Campaign:** please enter the required information below and then run the cell, check your output and set `ADD_NEW_CAMPAIGN` to `True` in the next cell prior to running. 

* **Updating a Campaign:**, please only enter the values you wish to update including the `campaign_year`, `campaign_interval` and `vessel code`  you wish to update. All previous values in the site file will be kept if not updated. One entered, run the cell, check your output and set `UPDATE_EXISTING_CAMPAIGN` to `True` in the next cell prior to running. 

* **Deleting a Transponder:** When deleting, only the `campaign_year`, `campaign_interval` and `vessel code` is needed. Once entered, run the cell, and set `DELETE_CAMPAIGN` to `True` in the next cell prior to running. 

In [ ]:
campaign = {}
# ----------------------- Update these values ----------------------- 

# -- Required information for new campaign --
# Generate the campaign name
campaign_year = ""              # Year of campaign - eg. 2024
campaign_interval = ""          # alphabetical interval of campaign - 1st: a, 2nd: b, etc..
vessel_code = ""                # 4 digit vessel code - this will also reference vessel metadata file

campaign['start'] = ""          # Start date of campaign, format '1900-01-01T00:00:00'
campaign['end'] = ""            # End date of campaign, format '1900-01-01T00:00:00'

campaign['type'] = ""           # type of campaign: deploy | measure 

# -- Optional: Enter information known about the people and vessels involved in the campaign --
campaign['launchVesselName'] = ""             # launch vessel name used in campaign
campaign['recoveryVesselName'] = ""           # recovery vessel name used in campaign
campaign['principalInvestigator'] = ""        # PI name 
campaign['cruiseName'] = ""                   # Name of cruise
campaign['technicianName'] = ""               # technician name
campaign['technicianContact'] = ""            # technician contact information (email/phone)


# ----------------------- Do not update code below ----------------------- 
campaign['name'], campaign['vesselCode'] = campaign_checks(campaign_year, campaign_interval, vessel_code)
print(json.dumps(campaign, indent=2))

#### Choose whether to add new, update, or delete the campaign you defined above

In [ ]:
# Set only one of the following to True
ADD_NEW_CAMPAIGN = True
UPDATE_EXISTING_CAMPAIGN = False
DELETE_CAMPAIGN = False

# ----------------------- Do not update code below -----------------------
site_class.run_component(component_type=TopLevelSiteGroups.CAMPAIGNS, component_metadata=campaign, 
                        add_new=ADD_NEW_CAMPAIGN, update=UPDATE_EXISTING_CAMPAIGN, delete=DELETE_CAMPAIGN)

## Campaign Survey

* **New Survey:** please enter the required information below and then run the cell, check your output and set `ADD_NEW_SURVEY` to `True` in the next cell prior to running. 
    * You do not need to enter a `survey_id` if entereing a new survey. The code will generate an ID for you.
    * The code below assumes the survey was part of an campaign and with a survey vessel entered above. If you wish to add to a different campaign / survey vessel, change the `survey_campaign_name` variable from the default.

* **Updating a Survey:**, please only enter the values you wish to update including the `survey_campaign_name` and `survey_id` you wish to update. All previous values in the site file will be kept if not updated. One entered, run the cell, check your output and set `UPDATE_EXISTING_SURVEY` to `True` in the next cell prior to running. 

* **Deleting a Surveyr:** When deleting, only the `survey_campaign_name` and `survey_id` is needed. Once entered, run the cell, and set `DELETE_SURVEY` to `True` in the next cell prior to running. 

In [ ]:
survey = {}
# ----------------------- Update these values ----------------------- 
# -- Required information for new survey --
survey_campaign_name = campaign['name']         # Change this if you didn't previously add a campaign or wish to add to a different campaign, example: 2024_A_GLDR

# ** Only add if updating or deleting an existing survey. Please check the current JSON file for the survey ID. **
survey['id'] = ""                       # campaign_name + interval e.g 2024_A_GLDR_1

survey['type'] = ""                     # e.g cirle drive | fixed point | mixed
survey['benchmarkIDs'] = ['']           # List of benchmark IDs in survey['IVB1-1', 'IVB1-2', 'IVB1-3']

survey['start'] = ""                    # Survey start time '1900-01-01T00:00:00'
survey['end'] = ""                      # Survey end time '1900-01-01T00:00:00'

# -- Optional Survey Information --
survey['commands'] = ""             # log of commands for a given survey
survey['notes'] = ""                # i.e any strange data flags

    
# ----------------------- Do not update code below -----------------------    
print(json.dumps(survey, indent=2))

#### Choose whether to add new, update, or delete the survey you defined above

In [ ]:
ADD_NEW_SURVEY = True
UPDATE_EXISTING_SURVEY = False
DELETE_SURVEY = False

# ----------------------- Do not update code below -----------------------
site_class.run_sub_component(component_type=TopLevelSiteGroups.CAMPAIGNS, component_name=survey_campaign_name,
                            sub_component_type=SubLevelSiteGroups.SURVEYS, sub_component_metadata=survey,
                            add_new=ADD_NEW_SURVEY, update=UPDATE_EXISTING_SURVEY, delete=DELETE_SURVEY)

***
## Other Site Specific Data
***
### Reference frames

Add reference frame name and start and end dates used (optional)

In [ ]:
reference_frame = {}
# ----------------------- Update these values ----------------------- 
# -- Required information for new reference frame  --
reference_frame['name'] = ""                # e.g ITRF2008
reference_frame['start'] = ""              # Start date of reference frame used, format: '1900-01-01T00:00:00'

# -- Optional information for reference frame --
reference_frame['end'] =  ""               # End date of reference frame used, format: 1900-01-01T00:00:00'

# ---------------------- Do not update code below ---------------------
print(json.dumps(reference_frame, indent=2))

#### Choose whether to add new, update, or delete the reference frame you defined above

In [ ]:
ADD_NEW_REFERENCE_FRAME = True
UPDATE_EXISTING_REFERENCE_FRAME = False
DELETE_REFERENCE_FRAME = False

# ---------------------- Do not update code below ---------------------
site_class.run_component(component_type=TopLevelSiteGroups.REFERENCE_FRAMES, component_metadata=reference_frame,
                        add_new=ADD_NEW_REFERENCE_FRAME, update=UPDATE_EXISTING_REFERENCE_FRAME, delete=DELETE_REFERENCE_FRAME)

***
### Additional Transponder Information 

For extra sensors and battery voltages, please add them below. We do not have the update/delete function and if needed, edit the json output directly.

#### Extra Sensors

In [ ]:
extra_sensor = {}
# ----------------------- Update these values -----------------------
benchmark_name = ""                                 # e.g "IVB1-1" - REQUIRED to add sensor 
transponder_address = ""                            # e.g 5209 - REQUIRED to add sensor
extra_sensor = ExtraSensors(
    type = '',                                      # e.g "pressure"
    serialNumber = '',                              # pressure sensor serial number
    model = ''                                      # pressure sensor model number    
)

#### Add the sensor you defined above

In [ ]:
site_class.run_sub_component(component_type=TopLevelSiteGroups.BENCHMARKS, component_name=benchmark_name,
                            sub_component_type=SubLevelSiteGroups.TRANSPONDERS, sub_component_metadata={'address': transponder_address, 'extraSensors': extra_sensor},
                            update=True)

#### Battery Voltages

In [ ]:
battery = {}
# ----------------------- Update these values -----------------------
# -- Required Battery Information --
benchmark_name = ""                     # e.g "IVB1-1"  
transponder_address = ""                # e.g 5209 
battery = BatteryVoltage(
    date = '2020-01-01T00:00:00',               # Observed Date & Time, Format: '1900-01-01T00:00:00'
    voltage = 12.0                              # e.g 12.0
)

#### Add the battery voltage

In [ ]:
site_class.run_sub_component(component_type=TopLevelSiteGroups.BENCHMARKS, component_name=benchmark_name,
                            sub_component_type=SubLevelSiteGroups.TRANSPONDERS, sub_component_metadata={'address': transponder_address, 'batteryVoltage': battery},
                            update=True)

## **4. Write out JSON file**
***
This step can be completed at any time.

* First check the contents...

In [ ]:
site_class.print_json()

### Export Site to file

In [ ]:
SITE_4_CHAR_ID = "" # 4 char site ID

if not SITE_4_CHAR_ID:
    raise ValueError("Please enter a 4 char site ID")

# Export site metadata to a json file
# Add date to the file name
date = datetime.now().strftime("%Y-%m-%d")
file_path = f"./{SITE_4_CHAR_ID + '.' + date}.json"                            # Export file path you wish to store
site_class.export_site(file_path)